In [ ]:
with open("genes.txt") as io:

    genes = [gene.strip() for gene in io.readlines()[1:]]

In [ ]:
import numpy as np

In [ ]:
# gene_values = np.full(len(genes), 1 / len(genes))
gene_values = np.random.normal(size=len(genes))

In [ ]:
import pandas as pd

In [ ]:
gene_x_sample = pd.DataFrame(
    {"Sample 1": gene_values, "Sample 2": gene_values * 10}, index=genes
)

In [ ]:
random_gene_set_genes = np.random.choice(genes, size=100, replace=False).tolist()

In [ ]:
import kraft

In [ ]:
gene_set_genes = {}

for gmt_file_path in [
    "/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c1.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c2.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c3.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c5.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c6.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/msigdb_v6.2/c7.all.v6.2.symbols.gmt",
    #     "/home/kwat/garden/data/gene_set/hippo_pathway.gmt",
    #     "/home/kwat/garden/data/gene_set/stem_cell.gmt",
    #     "/home/kwat/garden/data/gene_set/yap_signature.gmt",
]:

    gene_set_genes.update(kraft.read_gmt(gmt_file_path))